In [55]:
import pulp as plp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [56]:
g_df = pd.DataFrame({'region' : ['E', 'W', 'E', 'E', 'W'],
                     'power_g' : [15, 100, 32, 25, 70],
                     'cost_g' : [75, 15, 0, 42, 10]})
d_df = pd.DataFrame({'region' : ['E', 'E', 'E', 'E', 'W', 'E', 'W'],
                     'power_d' : [35, 23, 12, 38, 43, 16, 57],
                     'cost_d' : [65, 78, 10, 46, 63, 32, 50]})
# East Region
power_g_e = g_df[g_df['region']=='E'].values[:,1].T.tolist()
cost_g_e = g_df[g_df['region']=='E'].values[:,2].T.tolist()
power_d_e = d_df[d_df['region']=='E'].values[:,1].T.tolist()
cost_d_e = d_df[d_df['region']=='E'].values[:,2].T.tolist()

# West Region
power_g_w = g_df[g_df['region']=='W'].values[:,1].T.tolist()
cost_g_w = g_df[g_df['region']=='W'].values[:,2].T.tolist()
power_d_w = d_df[d_df['region']=='W'].values[:,1].T.tolist()
cost_d_w = d_df[d_df['region']=='W'].values[:,2].T.tolist()

# Interconnection
limit = 30


In [57]:
ng_e = len(power_g_e)
nd_e = len(power_d_e)

ng_w = len(power_g_w)
nd_w = len(power_d_w)

set_ng_e = range(1, ng_e + 1)
set_nd_e = range(1, nd_e + 1)

set_ng_w = range(1, ng_w + 1)
set_nd_w = range(1, nd_w + 1)

cg_e = {(i): cost_g_e[i-1] for i in set_ng_e}
pg_e = {(i): power_g_e[i-1] for i in set_ng_e}
cd_e = {(i): cost_d_e[i-1] for i in set_nd_e}
pd_e = {(i): power_d_e[i-1] for i in set_nd_e}

cg_w = {(i): cost_g_w[i-1] for i in set_ng_w}
pg_w = {(i): power_g_w[i-1] for i in set_ng_w}
cd_w = {(i): cost_d_w[i-1] for i in set_nd_w}
pd_w = {(i): power_d_w[i-1] for i in set_nd_w}

In [58]:
opt_model = plp.LpProblem(name="Model")

In [59]:
yg_e  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pg_e[i], name="yg_e_{}".format(i)) for i in set_ng_e}
yd_e  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pd_e[i], name="yd_e_{}".format(i)) for i in set_nd_e}

yg_w  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pg_w[i], name="yg_w_{}".format(i)) for i in set_ng_w}
yd_w  = {(i): plp.LpVariable(cat=plp.LpContinuous, lowBound=0, upBound=pd_w[i], name="yd_w_{}".format(i)) for i in set_nd_w}

theta = plp.LpVariable(cat=plp.LpContinuous, lowBound=-limit, upBound=limit, name="theta")

In [60]:
constraint_1 = plp.LpConstraint(
                 e=plp.lpSum(yd_w[j] for j in set_nd_w) - plp.lpSum(yg_w[j] for j in set_ng_w)-theta,
                 sense=plp.LpConstraintEQ,
                 rhs=0,
                 name="constraint_eq_1")

constraint_2 = plp.LpConstraint(
                 e=plp.lpSum(yd_e[j] for j in set_nd_e) - plp.lpSum(yg_e[j] for j in set_ng_e)+theta,
                 sense=plp.LpConstraintEQ,
                 rhs=0,
                 name="constraint_eq_2")

In [61]:
opt_model += constraint_1, "balance between supply and demand west"
opt_model += constraint_2, "balance between supply and demand east"

In [62]:
objective = plp.lpSum(yd_e[i]*cd_e[i] for i in set_nd_e) + plp.lpSum(yd_w[i]*cd_w[i] for i in set_nd_w) - (
            plp.lpSum(yg_e[i]*cg_e[i] for i in set_ng_e) + plp.lpSum(yg_w[i]*cg_w[i] for i in set_ng_w))

In [63]:
opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)

In [64]:
opt_model.solve()
#print(plp.LpStatus[opt_model.status])

1

In [65]:
opt_g_e_df = pd.DataFrame.from_dict(yg_e, orient="index", columns = ["variable_object"])
opt_g_w_df = pd.DataFrame.from_dict(yg_w, orient="index", columns = ["variable_object"])
opt_g_df = pd.concat([opt_g_e_df, opt_g_w_df], ignore_index = True)
opt_g_df["solution_value"] = opt_g_df["variable_object"].apply(lambda item: item.varValue)

opt_d_e_df = pd.DataFrame.from_dict(yd_e, orient="index", columns = ["variable_object"])
opt_d_w_df = pd.DataFrame.from_dict(yd_w, orient="index", columns = ["variable_object"])
opt_d_df = pd.concat([opt_d_e_df, opt_d_w_df], ignore_index = True)
opt_d_df["solution_value"] = opt_d_df["variable_object"].apply(lambda item: item.varValue)

In [66]:
opt_g_df

,variable_object,solution_value
0,yg_e_1,0.0
1,yg_e_2,32.0
2,yg_e_3,25.0
3,yg_w_1,60.0
4,yg_w_2,70.0


In [67]:
opt_d_df

,variable_object,solution_value
0,yd_e_1,35.0
1,yd_e_2,23.0
2,yd_e_3,0.0
3,yd_e_4,29.0
4,yd_e_5,0.0
5,yd_w_1,43.0
6,yd_w_2,57.0


In [68]:
for name, c in list(opt_model.constraints.items()):
    print(c.pi)

15.0
46.0


In [70]:
print(theta.varValue)

-30.0
